### Dependencies

In [2]:
from collections import OrderedDict
from typing import Dict, List, Optional, Tuple

import flwr as fl
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, random_split
from torchvision.datasets import CIFAR10

DEVICE = torch.device("cpu")  # Try "cuda" to train on GPU
print(f"Training on {DEVICE} using PyTorch {torch.__version__} and Flower {fl.__version__}")

Training on cpu using PyTorch 1.13.0 and Flower 1.1.0


## Data Loading

In [3]:
NUM_CLIENTS = 10

def load_datasets(num_clients: int):
    # Download and transform CIFAR-10 (train and test)
    transform = transforms.Compose(
      [transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]
    )
    trainset = CIFAR10("./dataset", train=True, download=True, transform=transform)
    testset = CIFAR10("./dataset", train=False, download=True, transform=transform)

    # Split training set into `num_clients` partitions to simulate different local datasets
    partition_size = len(trainset) // num_clients
    lengths = [partition_size] * num_clients
    datasets = random_split(trainset, lengths, torch.Generator().manual_seed(42))

    # Split each partition into train/val and create DataLoader
    trainloaders = []
    valloaders = []
    for ds in datasets:
        len_val = len(ds) // 10  # 10 % validation set
        len_train = len(ds) - len_val
        lengths = [len_train, len_val]
        ds_train, ds_val = random_split(ds, lengths, torch.Generator().manual_seed(42))
        trainloaders.append(DataLoader(ds_train, batch_size=32, shuffle=True))
        valloaders.append(DataLoader(ds_val, batch_size=32))
    testloader = DataLoader(testset, batch_size=32)
    return trainloaders, valloaders, testloader

trainloaders, valloaders, testloader = load_datasets(NUM_CLIENTS)

Files already downloaded and verified
Files already downloaded and verified


## Model training/evaluation

In [4]:
class Net(nn.Module):
    def __init__(self) -> None:
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


def get_parameters(net) -> List[np.ndarray]:
    return [val.cpu().numpy() for _, val in net.state_dict().items()]


def set_parameters(net, parameters: List[np.ndarray]):
    params_dict = zip(net.state_dict().keys(), parameters)
    state_dict = OrderedDict({k: torch.Tensor(v) for k, v in params_dict})
    net.load_state_dict(state_dict, strict=True)


def train(net, trainloader, epochs: int):
    """Train the network on the training set."""
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(net.parameters())
    net.train()
    for epoch in range(epochs):
        correct, total, epoch_loss = 0, 0, 0.0
        for images, labels in trainloader:
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            optimizer.zero_grad()
            outputs = net(images)
            loss = criterion(net(images), labels)
            loss.backward()
            optimizer.step()
            # Metrics
            epoch_loss += loss
            total += labels.size(0)
            correct += (torch.max(outputs.data, 1)[1] == labels).sum().item()
        epoch_loss /= len(trainloader.dataset)
        epoch_acc = correct / total
        print(f"Epoch {epoch+1}: train loss {epoch_loss}, accuracy {epoch_acc}")


def test(net, testloader):
    """Evaluate the network on the entire test set."""
    criterion = torch.nn.CrossEntropyLoss()
    correct, total, loss = 0, 0, 0.0
    net.eval()
    with torch.no_grad():
        for images, labels in testloader:
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            outputs = net(images)
            loss += criterion(outputs, labels).item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    loss /= len(testloader.dataset)
    accuracy = correct / total
    return loss, accuracy

## Flower clients

In [5]:
class FlowerClient(fl.client.NumPyClient):
    def __init__(self, cid, net, trainloader, valloader):
        self.cid = cid
        self.net = net
        self.trainloader = trainloader
        self.valloader = valloader

    def get_parameters(self, config):
        print(f"[Client {self.cid}] get_parameters")
        return get_parameters(self.net)

    def fit(self, parameters, config):
        print(f"[Client {self.cid}] fit, config: {config}")
        set_parameters(self.net, parameters)
        train(self.net, self.trainloader, epochs=1)
        return get_parameters(self.net), len(self.trainloader), {}

    def evaluate(self, parameters, config):
        print(f"[Client {self.cid}] evaluate, config: {config}")
        set_parameters(self.net, parameters)
        loss, accuracy = test(self.net, self.valloader)
        return float(loss), len(self.valloader), {"accuracy": float(accuracy)}


def client_fn(cid) -> FlowerClient:
    net = Net().to(DEVICE)
    trainloader = trainloaders[int(cid)]
    valloader = valloaders[int(cid)]
    return FlowerClient(cid, net, trainloader, valloader)

# Strategy customization

## Server-side parameter **initialization**

Flower, by default, initializes the global model by asking one random client for the initial parameters. In many cases, we want more control over parameter initialization though.
Flower therefore allows you to directly pass the initial parameters to the Strategy.

Passing initial_parameters to the FedAvg strategy prevents Flower from asking one of the clients for the initial parameters. If we look closely, we can see that the logs do not show any calls to the FlowerClient.get_parameters method.

In [6]:
# Create an instance of the model and get the parameters
params = get_parameters(Net())

# Pass parameters to the Strategy for server-side parameter initialization
strategy = fl.server.strategy.FedAvg(
    fraction_fit=0.3,
    fraction_evaluate=0.3,
    min_fit_clients=3,
    min_evaluate_clients=3,
    min_available_clients=NUM_CLIENTS,
    initial_parameters=fl.common.ndarrays_to_parameters(params),
)

# Start simulation
fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=NUM_CLIENTS,
    config=fl.server.ServerConfig(num_rounds=3),  # Just three rounds
    strategy=strategy,
)

INFO flower 2022-12-07 16:05:43,903 | app.py:140 | Starting Flower simulation, config: ServerConfig(num_rounds=3, round_timeout=None)
2022-12-07 16:05:45,318	INFO worker.py:1518 -- Started a local Ray instance.
INFO flower 2022-12-07 16:05:46,743 | app.py:174 | Flower VCE: Ray initialized with resources: {'node:127.0.0.1': 1.0, 'memory': 12397450855.0, 'CPU': 8.0, 'object_store_memory': 2147483648.0}
INFO flower 2022-12-07 16:05:46,745 | server.py:86 | Initializing global parameters
INFO flower 2022-12-07 16:05:46,745 | server.py:266 | Using initial parameters provided by strategy
INFO flower 2022-12-07 16:05:46,746 | server.py:88 | Evaluating initial parameters
INFO flower 2022-12-07 16:05:46,747 | server.py:101 | FL starting
DEBUG flower 2022-12-07 16:05:46,748 | server.py:215 | fit_round 1: strategy sampled 3 clients (out of 10)


(launch_and_fit pid=9801) [Client 6] fit, config: {}
(launch_and_fit pid=9798) [Client 5] fit, config: {}
(launch_and_fit pid=9800) [Client 1] fit, config: {}


DEBUG flower 2022-12-07 16:05:54,022 | server.py:229 | fit_round 1 received 3 results and 0 failures
WARNING flower 2022-12-07 16:05:54,030 | fedavg.py:242 | No fit_metrics_aggregation_fn provided
DEBUG flower 2022-12-07 16:05:54,031 | server.py:165 | evaluate_round 1: strategy sampled 3 clients (out of 10)


(launch_and_fit pid=9801) Epoch 1: train loss 0.06420566886663437, accuracy 0.2328888888888889
(launch_and_fit pid=9798) Epoch 1: train loss 0.0642307847738266, accuracy 0.23866666666666667
(launch_and_fit pid=9800) Epoch 1: train loss 0.06421255320310593, accuracy 0.2311111111111111


DEBUG flower 2022-12-07 16:05:57,263 | server.py:179 | evaluate_round 1 received 3 results and 0 failures
WARNING flower 2022-12-07 16:05:57,264 | fedavg.py:273 | No evaluate_metrics_aggregation_fn provided
DEBUG flower 2022-12-07 16:05:57,264 | server.py:215 | fit_round 2: strategy sampled 3 clients (out of 10)


...

(launch_and_evaluate pid=9801) [Client 2] evaluate, config: {}
(launch_and_evaluate pid=9798) [Client 0] evaluate, config: {}
(launch_and_evaluate pid=9800) [Client 6] evaluate, config: {}
(launch_and_fit pid=9801) [Client 5] fit, config: {}
(launch_and_fit pid=9798) [Client 0] fit, config: {}
(launch_and_fit pid=9800) [Client 8] fit, config: {}


DEBUG flower 2022-12-07 16:06:11,300 | server.py:229 | fit_round 3 received 3 results and 0 failures
DEBUG flower 2022-12-07 16:06:11,308 | server.py:165 | evaluate_round 3: strategy sampled 3 clients (out of 10)


(launch_and_fit pid=9801) Epoch 1: train loss 0.052973777055740356, accuracy 0.38355555555555554
(launch_and_fit pid=9798) Epoch 1: train loss 0.052766550332307816, accuracy 0.38177777777777777
(launch_and_fit pid=9800) Epoch 1: train loss 0.05191192403435707, accuracy 0.39644444444444443
(launch_and_evaluate pid=9798) [Client 0] evaluate, config: {}
(launch_and_evaluate pid=9801) [Client 5] evaluate, config: {}
(launch_and_evaluate pid=9800) [Client 1] evaluate, config: {}


DEBUG flower 2022-12-07 16:06:14,930 | server.py:179 | evaluate_round 3 received 3 results and 0 failures
INFO flower 2022-12-07 16:06:14,931 | server.py:144 | FL finished in 28.182679120000103
INFO flower 2022-12-07 16:06:14,932 | app.py:192 | app_fit: losses_distributed [(1, 0.05941859984397888), (2, 0.05397985498110453), (3, 0.050876362244288126)]
INFO flower 2022-12-07 16:06:14,933 | app.py:193 | app_fit: metrics_distributed {}
INFO flower 2022-12-07 16:06:14,934 | app.py:194 | app_fit: losses_centralized []
INFO flower 2022-12-07 16:06:14,935 | app.py:195 | app_fit: metrics_centralized {}


History (loss, distributed):
	round 1: 0.05941859984397888
	round 2: 0.05397985498110453
	round 3: 0.050876362244288126

## Starting with a customized strategy
You can choose FedAvg or FedAdagrad or others strategies

In [ ]:
# Create FedAdam strategy
strategy=fl.server.strategy.FedAdagrad(
    fraction_fit=0.3,
    fraction_evaluate=0.3,
    min_fit_clients=3,
    min_evaluate_clients=3,
    min_available_clients=NUM_CLIENTS,
    initial_parameters=fl.common.ndarrays_to_parameters(get_parameters(Net())),
)

# Start simulation
fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=NUM_CLIENTS,
    config=fl.server.ServerConfig(num_rounds=3),  # Just three rounds
    strategy=strategy,
)

## Server-side parameter **evaluation**

**Centralized Evaluation (or server-side evaluation)**
is conceptually simple: it works the same way that evaluation in centralized machine learning does.
If there is a **server-side dataset that can be used for evaluation purposes**, then that’s great.
We can evaluate the newly aggregated model after each round of training without having to send the model to clients.
We’re also fortunate in the sense that our entire evaluation dataset is available at all times.

**Federated Evaluation (or client-side evaluation)**
 is more complex, but also more powerful: it doesn’t require a centralized dataset and allows us to evaluate models over a larger set of data, which often yields more realistic evaluation results.
 In fact, many scenarios require us to use Federated Evaluation if we want to get representative evaluation results at all.
 But this power comes at a cost: once we start to evaluate on the client side, **we should be aware that our evaluation dataset can change over consecutive rounds of learning if those clients are not always available**.
 Moreover, **the dataset held by each client can also change over consecutive rounds**.
 This can lead to evaluation results that are not stable, so even if we would not change the model, we’d see our evaluation results fluctuate over consecutive rounds.

In [7]:
# The `evaluate` function will be by Flower called after every round
def evaluate(
    server_round: int, parameters: fl.common.NDArrays, config: Dict[str, fl.common.Scalar]
) -> Optional[Tuple[float, Dict[str, fl.common.Scalar]]]:
    net = Net()
    valloader = valloaders[0]
    set_parameters(net, parameters)  # Update model with the latest parameters
    loss, accuracy = test(net, valloader)
    print(f"Server-side evaluation loss {loss} / accuracy {accuracy}")
    return loss, {"accuracy": accuracy}

In [8]:
strategy = fl.server.strategy.FedAvg(
    fraction_fit=0.3,
    fraction_evaluate=0.3,
    min_fit_clients=3,
    min_evaluate_clients=3,
    min_available_clients=NUM_CLIENTS,
    initial_parameters=fl.common.ndarrays_to_parameters(get_parameters(Net())),
    evaluate_fn=evaluate,  # Pass the evaluation function
)

fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=NUM_CLIENTS,
    config=fl.server.ServerConfig(num_rounds=3),  # Just three rounds
    strategy=strategy,
)

INFO flower 2022-12-07 16:31:15,926 | app.py:140 | Starting Flower simulation, config: ServerConfig(num_rounds=3, round_timeout=None)
2022-12-07 16:31:19,203	INFO worker.py:1518 -- Started a local Ray instance.
INFO flower 2022-12-07 16:31:20,582 | app.py:174 | Flower VCE: Ray initialized with resources: {'object_store_memory': 2147483648.0, 'CPU': 8.0, 'node:127.0.0.1': 1.0, 'memory': 14397673063.0}
INFO flower 2022-12-07 16:31:20,583 | server.py:86 | Initializing global parameters
INFO flower 2022-12-07 16:31:20,584 | server.py:266 | Using initial parameters provided by strategy
INFO flower 2022-12-07 16:31:20,585 | server.py:88 | Evaluating initial parameters
INFO flower 2022-12-07 16:31:20,752 | server.py:91 | initial parameters (loss, other metrics): 0.07370910120010377, {'accuracy': 0.098}
INFO flower 2022-12-07 16:31:20,753 | server.py:101 | FL starting
DEBUG flower 2022-12-07 16:31:20,754 | server.py:215 | fit_round 1: strategy sampled 3 clients (out of 10)


Server-side evaluation loss 0.07370910120010377 / accuracy 0.098
(launch_and_fit pid=9924) [Client 8] fit, config: {}
(launch_and_fit pid=9922) [Client 0] fit, config: {}
(launch_and_fit pid=9926) [Client 6] fit, config: {}


DEBUG flower 2022-12-07 16:31:27,854 | server.py:229 | fit_round 1 received 3 results and 0 failures
WARNING flower 2022-12-07 16:31:27,861 | fedavg.py:242 | No fit_metrics_aggregation_fn provided
INFO flower 2022-12-07 16:31:28,007 | server.py:116 | fit progress: (1, 0.06255912685394287, {'accuracy': 0.318}, 7.253054238999994)
DEBUG flower 2022-12-07 16:31:28,007 | server.py:165 | evaluate_round 1: strategy sampled 3 clients (out of 10)


(launch_and_fit pid=9924) Epoch 1: train loss 0.0646289512515068, accuracy 0.23244444444444445
(launch_and_fit pid=9922) Epoch 1: train loss 0.06527634710073471, accuracy 0.22933333333333333
(launch_and_fit pid=9926) Epoch 1: train loss 0.06476641446352005, accuracy 0.2388888888888889
Server-side evaluation loss 0.06255912685394287 / accuracy 0.318


DEBUG flower 2022-12-07 16:31:31,141 | server.py:179 | evaluate_round 1 received 3 results and 0 failures
WARNING flower 2022-12-07 16:31:31,142 | fedavg.py:273 | No evaluate_metrics_aggregation_fn provided
DEBUG flower 2022-12-07 16:31:31,142 | server.py:215 | fit_round 2: strategy sampled 3 clients (out of 10)


...

(launch_and_fit pid=9922) [Client 2] fit, config: {}
(launch_and_fit pid=9924) [Client 3] fit, config: {}
(launch_and_fit pid=9926) [Client 6] fit, config: {}


DEBUG flower 2022-12-07 16:31:46,271 | server.py:229 | fit_round 3 received 3 results and 0 failures
INFO flower 2022-12-07 16:31:46,423 | server.py:116 | fit progress: (3, 0.05349695038795471, {'accuracy': 0.402}, 25.669028311000147)
DEBUG flower 2022-12-07 16:31:46,424 | server.py:165 | evaluate_round 3: strategy sampled 3 clients (out of 10)


(launch_and_fit pid=9922) Epoch 1: train loss 0.054621271789073944, accuracy 0.3571111111111111
(launch_and_fit pid=9926) Epoch 1: train loss 0.05538584291934967, accuracy 0.35
(launch_and_fit pid=9924) Epoch 1: train loss 0.05476818233728409, accuracy 0.35844444444444445
Server-side evaluation loss 0.05349695038795471 / accuracy 0.402
(launch_and_evaluate pid=9924) [Client 3] evaluate, config: {}
(launch_and_evaluate pid=9926) [Client 9] evaluate, config: {}
(launch_and_evaluate pid=9922) [Client 8] evaluate, config: {}


DEBUG flower 2022-12-07 16:31:49,774 | server.py:179 | evaluate_round 3 received 3 results and 0 failures
INFO flower 2022-12-07 16:31:49,775 | server.py:144 | FL finished in 29.021321522000108
INFO flower 2022-12-07 16:31:49,776 | app.py:192 | app_fit: losses_distributed [(1, 0.06284476073582967), (2, 0.0545183969338735), (3, 0.052845369974772134)]
INFO flower 2022-12-07 16:31:49,777 | app.py:193 | app_fit: metrics_distributed {}
INFO flower 2022-12-07 16:31:49,778 | app.py:194 | app_fit: losses_centralized [(0, 0.07370910120010377), (1, 0.06255912685394287), (2, 0.05607489824295044), (3, 0.05349695038795471)]
INFO flower 2022-12-07 16:31:49,779 | app.py:195 | app_fit: metrics_centralized {'accuracy': [(0, 0.098), (1, 0.318), (2, 0.372), (3, 0.402)]}


History (loss, distributed):
	round 1: 0.06284476073582967
	round 2: 0.0545183969338735
	round 3: 0.052845369974772134
History (loss, centralized):
	round 0: 0.07370910120010377
	round 1: 0.06255912685394287
	round 2: 0.05607489824295044
	round 3: 0.05349695038795471
History (metrics, centralized):
{'accuracy': [(0, 0.098), (1, 0.318), (2, 0.372), (3, 0.402)]}

## Sending/receiving arbitrary values to/from clients
In some situations, we want to configure client-side execution (trainig, evaluation) from the server-side.  (i.e. the number of local epochs)
 Flower provides a way to send configuration values from the server to the clients using a dictionary.
 Let’s look at an example where the clients receive values from the server through the config parameter in fit (config is also available in evaluate).
 The fit method receives the configuration dictionary through the config parameter and can then read values from this dictionary.

In [9]:
class FlowerClient(fl.client.NumPyClient):
    def __init__(self, cid, net, trainloader, valloader):
        self.cid = cid
        self.net = net
        self.trainloader = trainloader
        self.valloader = valloader

    def get_parameters(self, config):
        print(f"[Client {self.cid}] get_parameters")
        return get_parameters(self.net)

    def fit(self, parameters, config):
        # Read values from config
        server_round = config["server_round"]
        local_epochs = config["local_epochs"]

        # Use values provided by the config
        print(f"[Client {self.cid}, round {server_round}] fit, config: {config}")
        set_parameters(self.net, parameters)
        train(self.net, self.trainloader, epochs=local_epochs)
        return get_parameters(self.net), len(self.trainloader), {}

    def evaluate(self, parameters, config):
        print(f"[Client {self.cid}] evaluate, config: {config}")
        set_parameters(self.net, parameters)
        loss, accuracy = test(self.net, self.valloader)
        return float(loss), len(self.valloader), {"accuracy": float(accuracy)}


def client_fn(cid) -> FlowerClient:
    net = Net().to(DEVICE)
    trainloader = trainloaders[int(cid)]
    valloader = valloaders[int(cid)]
    return FlowerClient(cid, net, trainloader, valloader)

In [10]:
def fit_config(server_round: int):
    """Return training configuration dict for each round.

    Perform two rounds of training with one local epoch, increase to two local
    epochs afterwards.
    """
    config = {
        "server_round": server_round,  # The current round of federated learning
        "local_epochs": 1 if server_round < 2 else 2,  #
    }
    return config

As we can see, the client logs now include the current round of federated learning (which they read from the config dictionary).
We can also configure local training to run for one epoch during the first and second round of federated learning, and then for two epochs during the third round.



In [11]:
strategy = fl.server.strategy.FedAvg(
    fraction_fit=0.3,
    fraction_evaluate=0.3,
    min_fit_clients=3,
    min_evaluate_clients=3,
    min_available_clients=NUM_CLIENTS,
    initial_parameters=fl.common.ndarrays_to_parameters(get_parameters(Net())),
    evaluate_fn=evaluate,
    on_fit_config_fn=fit_config,  # Pass the fit_config function
)

fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=NUM_CLIENTS,
    config=fl.server.ServerConfig(num_rounds=3),  # Just three rounds
    strategy=strategy,    # keeps the config
)

INFO flower 2022-12-07 16:47:13,131 | app.py:140 | Starting Flower simulation, config: ServerConfig(num_rounds=3, round_timeout=None)
2022-12-07 16:47:17,633	INFO worker.py:1518 -- Started a local Ray instance.
INFO flower 2022-12-07 16:47:18,889 | app.py:174 | Flower VCE: Ray initialized with resources: {'object_store_memory': 2147483648.0, 'memory': 15349331968.0, 'CPU': 8.0, 'node:127.0.0.1': 1.0}
INFO flower 2022-12-07 16:47:18,891 | server.py:86 | Initializing global parameters
INFO flower 2022-12-07 16:47:18,892 | server.py:266 | Using initial parameters provided by strategy
INFO flower 2022-12-07 16:47:18,892 | server.py:88 | Evaluating initial parameters
INFO flower 2022-12-07 16:47:19,075 | server.py:91 | initial parameters (loss, other metrics): 0.07371789598464966, {'accuracy': 0.098}
INFO flower 2022-12-07 16:47:19,076 | server.py:101 | FL starting
DEBUG flower 2022-12-07 16:47:19,077 | server.py:215 | fit_round 1: strategy sampled 3 clients (out of 10)


Server-side evaluation loss 0.07371789598464966 / accuracy 0.098
(launch_and_fit pid=9999) [Client 2, round 1] fit, config: {'server_round': 1, 'local_epochs': 1}
(launch_and_fit pid=9997) [Client 8, round 1] fit, config: {'server_round': 1, 'local_epochs': 1}
(launch_and_fit pid=10000) [Client 6, round 1] fit, config: {'server_round': 1, 'local_epochs': 1}


DEBUG flower 2022-12-07 16:47:26,174 | server.py:229 | fit_round 1 received 3 results and 0 failures
WARNING flower 2022-12-07 16:47:26,181 | fedavg.py:242 | No fit_metrics_aggregation_fn provided
INFO flower 2022-12-07 16:47:26,323 | server.py:116 | fit progress: (1, 0.06295389723777771, {'accuracy': 0.256}, 7.246437159999914)
DEBUG flower 2022-12-07 16:47:26,324 | server.py:165 | evaluate_round 1: strategy sampled 3 clients (out of 10)


(launch_and_fit pid=9999) Epoch 1: train loss 0.06530040502548218, accuracy 0.2248888888888889
(launch_and_fit pid=9997) Epoch 1: train loss 0.06531316787004471, accuracy 0.23222222222222222
(launch_and_fit pid=10000) Epoch 1: train loss 0.06542231887578964, accuracy 0.23066666666666666
Server-side evaluation loss 0.06295389723777771 / accuracy 0.256


DEBUG flower 2022-12-07 16:47:29,531 | server.py:179 | evaluate_round 1 received 3 results and 0 failures
WARNING flower 2022-12-07 16:47:29,532 | fedavg.py:273 | No evaluate_metrics_aggregation_fn provided
DEBUG flower 2022-12-07 16:47:29,533 | server.py:215 | fit_round 2: strategy sampled 3 clients (out of 10)


...

DEBUG flower 2022-12-07 16:47:48,466 | server.py:229 | fit_round 3 received 3 results and 0 failures
INFO flower 2022-12-07 16:47:48,627 | server.py:116 | fit progress: (3, 0.04944743013381958, {'accuracy': 0.43}, 29.550311821999912)
DEBUG flower 2022-12-07 16:47:48,628 | server.py:165 | evaluate_round 3: strategy sampled 3 clients (out of 10)


(launch_and_fit pid=9999) Epoch 2: train loss 0.0485663115978241, accuracy 0.44133333333333336
(launch_and_fit pid=9997) Epoch 2: train loss 0.04866097494959831, accuracy 0.436
(launch_and_fit pid=10000) Epoch 2: train loss 0.048769816756248474, accuracy 0.4211111111111111
Server-side evaluation loss 0.04944743013381958 / accuracy 0.43


DEBUG flower 2022-12-07 16:47:52,104 | server.py:179 | evaluate_round 3 received 3 results and 0 failures
INFO flower 2022-12-07 16:47:52,105 | server.py:144 | FL finished in 33.0282801860003
INFO flower 2022-12-07 16:47:52,106 | app.py:192 | app_fit: losses_distributed [(1, 0.06301429986953735), (2, 0.05221937688191732), (3, 0.04907084226608276)]
INFO flower 2022-12-07 16:47:52,107 | app.py:193 | app_fit: metrics_distributed {}
INFO flower 2022-12-07 16:47:52,108 | app.py:194 | app_fit: losses_centralized [(0, 0.07371789598464966), (1, 0.06295389723777771), (2, 0.052439891815185546), (3, 0.04944743013381958)]
INFO flower 2022-12-07 16:47:52,109 | app.py:195 | app_fit: metrics_centralized {'accuracy': [(0, 0.098), (1, 0.256), (2, 0.404), (3, 0.43)]}


(launch_and_evaluate pid=9999) [Client 1] evaluate, config: {}
(launch_and_evaluate pid=9997) [Client 4] evaluate, config: {}
(launch_and_evaluate pid=10000) [Client 9] evaluate, config: {}


History (loss, distributed):
	round 1: 0.06301429986953735
	round 2: 0.05221937688191732
	round 3: 0.04907084226608276
History (loss, centralized):
	round 0: 0.07371789598464966
	round 1: 0.06295389723777771
	round 2: 0.052439891815185546
	round 3: 0.04944743013381958
History (metrics, centralized):
{'accuracy': [(0, 0.098), (1, 0.256), (2, 0.404), (3, 0.43)]}

## Scaling federated learning


In [13]:
NUM_CLIENTS = 1000

trainloaders, valloaders, testloader = load_datasets(NUM_CLIENTS)

Files already downloaded and verified
Files already downloaded and verified


In [14]:
def fit_config(server_round: int):
    config = {
        "server_round": server_round,
        "local_epochs": 3,
    }
    return config

strategy = fl.server.strategy.FedAvg(
    fraction_fit=0.025,  # Train on 25 clients (each round)
    fraction_evaluate=0.05,  # Evaluate on 50 clients (each round)
    min_fit_clients=20,
    min_evaluate_clients=40,
    min_available_clients=NUM_CLIENTS,
    initial_parameters=fl.common.ndarrays_to_parameters(get_parameters(Net())),
    on_fit_config_fn=fit_config,
)

fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=NUM_CLIENTS,
    config=fl.server.ServerConfig(num_rounds=3),  # Just three rounds
    strategy=strategy,
)

INFO flower 2022-12-07 16:58:14,371 | app.py:140 | Starting Flower simulation, config: ServerConfig(num_rounds=3, round_timeout=None)
2022-12-07 16:58:17,540	INFO worker.py:1518 -- Started a local Ray instance.
INFO flower 2022-12-07 16:58:18,989 | app.py:174 | Flower VCE: Ray initialized with resources: {'node:127.0.0.1': 1.0, 'CPU': 8.0, 'memory': 15512628429.0, 'object_store_memory': 2147483648.0}
INFO flower 2022-12-07 16:58:18,993 | server.py:86 | Initializing global parameters
INFO flower 2022-12-07 16:58:18,993 | server.py:266 | Using initial parameters provided by strategy
INFO flower 2022-12-07 16:58:18,994 | server.py:88 | Evaluating initial parameters
INFO flower 2022-12-07 16:58:18,995 | server.py:101 | FL starting
DEBUG flower 2022-12-07 16:58:18,995 | server.py:215 | fit_round 1: strategy sampled 25 clients (out of 1000)


(launch_and_fit pid=10063) [Client 337, round 1] fit, config: {'server_round': 1, 'local_epochs': 3}
(launch_and_fit pid=10064) [Client 496, round 1] fit, config: {'server_round': 1, 'local_epochs': 3}
(launch_and_fit pid=10060) [Client 645, round 1] fit, config: {'server_round': 1, 'local_epochs': 3}
(launch_and_fit pid=10061) [Client 913, round 1] fit, config: {'server_round': 1, 'local_epochs': 3}
(launch_and_fit pid=10062) [Client 82, round 1] fit, config: {'server_round': 1, 'local_epochs': 3}
(launch_and_fit pid=10059) [Client 793, round 1] fit, config: {'server_round': 1, 'local_epochs': 3}
(launch_and_fit pid=10057) [Client 535, round 1] fit, config: {'server_round': 1, 'local_epochs': 3}
(launch_and_fit pid=10058) [Client 884, round 1] fit, config: {'server_round': 1, 'local_epochs': 3}
(launch_and_fit pid=10063) Epoch 1: train loss 0.10315325111150742, accuracy 0.022222222222222223
(launch_and_fit pid=10063) Epoch 2: train loss 0.10189681500196457, accuracy 0.1333333333333333

(raylet) Spilled 2213 MiB, 34 objects, write throughput 409 MiB/s. Set RAY_verbose_spill_logs=0 to disable this message.


(launch_and_fit pid=10063) Epoch 1: train loss 0.10222136974334717, accuracy 0.1111111111111111
(launch_and_fit pid=10063) Epoch 2: train loss 0.10092472285032272, accuracy 0.2
(launch_and_fit pid=10061) [Client 689, round 1] fit, config: {'server_round': 1, 'local_epochs': 3}
(launch_and_fit pid=10062) [Client 145, round 1] fit, config: {'server_round': 1, 'local_epochs': 3}
(launch_and_fit pid=10063) Epoch 3: train loss 0.10053155571222305, accuracy 0.3111111111111111
(launch_and_fit pid=10061) Epoch 1: train loss 0.1025557592511177, accuracy 0.08888888888888889
(launch_and_fit pid=10061) Epoch 2: train loss 0.1017594262957573, accuracy 0.2
(launch_and_fit pid=10062) Epoch 1: train loss 0.10220275819301605, accuracy 0.15555555555555556
(launch_and_fit pid=10062) Epoch 2: train loss 0.10089632868766785, accuracy 0.2222222222222222


DEBUG flower 2022-12-07 16:58:51,062 | server.py:229 | fit_round 1 received 25 results and 0 failures


...

DEBUG flower 2022-12-07 17:01:39,963 | server.py:229 | fit_round 3 received 25 results and 0 failures
DEBUG flower 2022-12-07 17:01:40,016 | server.py:165 | evaluate_round 3: strategy sampled 50 clients (out of 1000)


(launch_and_fit pid=10064) [Client 284, round 3] fit, config: {'server_round': 3, 'local_epochs': 3}
(launch_and_fit pid=10064) Epoch 1: train loss 0.10157642513513565, accuracy 0.17777777777777778
(launch_and_fit pid=10060) [Client 173, round 3] fit, config: {'server_round': 3, 'local_epochs': 3}
(launch_and_fit pid=10060) Epoch 1: train loss 0.1009516790509224, accuracy 0.28888888888888886
(launch_and_fit pid=10059) Epoch 3: train loss 0.09708540141582489, accuracy 0.28888888888888886
(launch_and_fit pid=10064) Epoch 2: train loss 0.09824725240468979, accuracy 0.3333333333333333
(launch_and_fit pid=10064) Epoch 3: train loss 0.0963376984000206, accuracy 0.26666666666666666
(launch_and_fit pid=10060) Epoch 2: train loss 0.09925661981105804, accuracy 0.28888888888888886
(launch_and_fit pid=10060) Epoch 3: train loss 0.09538258612155914, accuracy 0.28888888888888886
(launch_and_evaluate pid=10064) [Client 910] evaluate, config: {}
(launch_and_evaluate pid=10060) [Client 472] evaluate, c

DEBUG flower 2022-12-07 17:02:31,921 | server.py:179 | evaluate_round 3 received 50 results and 0 failures
INFO flower 2022-12-07 17:02:31,922 | server.py:144 | FL finished in 252.926321379
INFO flower 2022-12-07 17:02:31,923 | app.py:192 | app_fit: losses_distributed [(1, 0.46027768898010274), (2, 0.4553672752380373), (3, 0.45258576393127425)]
INFO flower 2022-12-07 17:02:31,924 | app.py:193 | app_fit: metrics_distributed {}
INFO flower 2022-12-07 17:02:31,925 | app.py:194 | app_fit: losses_centralized []
INFO flower 2022-12-07 17:02:31,925 | app.py:195 | app_fit: metrics_centralized {}


(launch_and_evaluate pid=10061) [Client 183] evaluate, config: {}
(launch_and_evaluate pid=10063) [Client 392] evaluate, config: {}
(launch_and_evaluate pid=10064) [Client 925] evaluate, config: {}
(launch_and_evaluate pid=10060) [Client 135] evaluate, config: {}
(launch_and_evaluate pid=10059) [Client 506] evaluate, config: {}


History (loss, distributed):
	round 1: 0.46027768898010274
	round 2: 0.4553672752380373
	round 3: 0.45258576393127425

# STRATEGY


### Preparation

In [15]:
NUM_CLIENTS = 10

def load_datasets(num_clients: int):
    # Download and transform CIFAR-10 (train and test)
    transform = transforms.Compose(
      [transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]
    )
    trainset = CIFAR10("./dataset", train=True, download=True, transform=transform)
    testset = CIFAR10("./dataset", train=False, download=True, transform=transform)

    # Split training set into `num_clients` partitions to simulate different local datasets
    partition_size = len(trainset) // num_clients
    lengths = [partition_size] * num_clients
    datasets = random_split(trainset, lengths, torch.Generator().manual_seed(42))

    # Split each partition into train/val and create DataLoader
    trainloaders = []
    valloaders = []
    for ds in datasets:
        len_val = len(ds) // 10  # 10 % validation set
        len_train = len(ds) - len_val
        lengths = [len_train, len_val]
        ds_train, ds_val = random_split(ds, lengths, torch.Generator().manual_seed(42))
        trainloaders.append(DataLoader(ds_train, batch_size=32, shuffle=True))
        valloaders.append(DataLoader(ds_val, batch_size=32))
    testloader = DataLoader(testset, batch_size=32)
    return trainloaders, valloaders, testloader

trainloaders, valloaders, testloader = load_datasets(NUM_CLIENTS)

Files already downloaded and verified
Files already downloaded and verified


In [16]:
class Net(nn.Module):
    def __init__(self) -> None:
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


def get_parameters(net) -> List[np.ndarray]:
    return [val.cpu().numpy() for _, val in net.state_dict().items()]


def set_parameters(net, parameters: List[np.ndarray]):
    params_dict = zip(net.state_dict().keys(), parameters)
    state_dict = OrderedDict({k: torch.Tensor(v) for k, v in params_dict})
    net.load_state_dict(state_dict, strict=True)


def train(net, trainloader, epochs: int):
    """Train the network on the training set."""
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(net.parameters())
    net.train()
    for epoch in range(epochs):
        correct, total, epoch_loss = 0, 0, 0.0
        for images, labels in trainloader:
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            optimizer.zero_grad()
            outputs = net(images)
            loss = criterion(net(images), labels)
            loss.backward()
            optimizer.step()
            # Metrics
            epoch_loss += loss
            total += labels.size(0)
            correct += (torch.max(outputs.data, 1)[1] == labels).sum().item()
        epoch_loss /= len(trainloader.dataset)
        epoch_acc = correct / total
        print(f"Epoch {epoch+1}: train loss {epoch_loss}, accuracy {epoch_acc}")


def test(net, testloader):
    """Evaluate the network on the entire test set."""
    criterion = torch.nn.CrossEntropyLoss()
    correct, total, loss = 0, 0, 0.0
    net.eval()
    with torch.no_grad():
        for images, labels in testloader:
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            outputs = net(images)
            loss += criterion(outputs, labels).item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    loss /= len(testloader.dataset)
    accuracy = correct / total
    return loss, accuracy

In [17]:
class FlowerClient(fl.client.NumPyClient):
    def __init__(self, cid, net, trainloader, valloader):
        self.cid = cid
        self.net = net
        self.trainloader = trainloader
        self.valloader = valloader

    def get_parameters(self, config):
        print(f"[Client {self.cid}] get_parameters")
        return get_parameters(self.net)

    def fit(self, parameters, config):
        print(f"[Client {self.cid}] fit, config: {config}")
        set_parameters(self.net, parameters)
        train(self.net, self.trainloader, epochs=1)
        return get_parameters(self.net), len(self.trainloader), {}

    def evaluate(self, parameters, config):
        print(f"[Client {self.cid}] evaluate, config: {config}")
        set_parameters(self.net, parameters)
        loss, accuracy = test(self.net, self.valloader)
        return float(loss), len(self.valloader), {"accuracy": float(accuracy)}


def client_fn(cid) -> FlowerClient:
    net = Net().to(DEVICE)
    trainloader = trainloaders[int(cid)]
    valloader = valloaders[int(cid)]
    return FlowerClient(cid, net, trainloader, valloader)

## Build a Strategy from scratch

In [18]:
from typing import Callable, Union

from flwr.common import (
    EvaluateIns,
    EvaluateRes,
    FitIns,
    FitRes,
    MetricsAggregationFn,
    NDArrays,
    Parameters,
    Scalar,
    ndarrays_to_parameters,
    parameters_to_ndarrays,
)
from flwr.server.client_manager import ClientManager
from flwr.server.client_proxy import ClientProxy


In [19]:


class FedCustom(fl.server.strategy.Strategy):
    def __repr__(self) -> str:
        return "FedCustom"

    def initialize_parameters(
        self, client_manager: ClientManager
    ) -> Optional[Parameters]:
        """Initialize global model parameters."""
        net = Net()
        ndarrays = get_parameters(net)
        return fl.common.ndarrays_to_parameters(ndarrays)

    def configure_fit(
        self, server_round: int, parameters: Parameters, client_manager: ClientManager
    ) -> List[Tuple[ClientProxy, FitIns]]:
        """Configure the next round of training."""

        # TODO WIP - add implementation

        return []

    def aggregate_fit(
        self,
        server_round: int,
        results: List[Tuple[ClientProxy, FitRes]],
        failures: List[Union[Tuple[ClientProxy, FitRes], BaseException]],
    ) -> Tuple[Optional[Parameters], Dict[str, Scalar]]:
        """Aggregate fit results using weighted average."""

        # TODO WIP - add implementation

        return None, {}

    def configure_evaluate(
        self, server_round: int, parameters: Parameters, client_manager: ClientManager
    ) -> List[Tuple[ClientProxy, EvaluateIns]]:
        """Configure the next round of evaluation."""

        # TODO WIP - add implementation

        return []

    def aggregate_evaluate(
        self,
        server_round: int,
        results: List[Tuple[ClientProxy, EvaluateRes]],
        failures: List[Union[Tuple[ClientProxy, EvaluateRes], BaseException]],
    ) -> Tuple[Optional[float], Dict[str, Scalar]]:
        """Aggregate evaluation losses using weighted average."""

        # TODO WIP - add implementation

        return None, {}

    def evaluate(
        self, server_round: int, parameters: Parameters
    ) -> Optional[Tuple[float, Dict[str, Scalar]]]:
        """Evaluate model parameters using an evaluation function."""

        # TODO WIP - add implementation

        return None

In [20]:
fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=2,
    config=fl.server.ServerConfig(num_rounds=3),
    strategy=FedCustom(),  # <-- pass the new strategy here
)

INFO flower 2022-12-07 17:14:35,130 | app.py:140 | Starting Flower simulation, config: ServerConfig(num_rounds=3, round_timeout=None)
2022-12-07 17:14:39,934	INFO worker.py:1518 -- Started a local Ray instance.
INFO flower 2022-12-07 17:14:41,331 | app.py:174 | Flower VCE: Ray initialized with resources: {'memory': 17911634330.0, 'object_store_memory': 2147483648.0, 'node:127.0.0.1': 1.0, 'CPU': 8.0}
INFO flower 2022-12-07 17:14:41,333 | server.py:86 | Initializing global parameters
INFO flower 2022-12-07 17:14:41,336 | server.py:266 | Using initial parameters provided by strategy
INFO flower 2022-12-07 17:14:41,337 | server.py:88 | Evaluating initial parameters
INFO flower 2022-12-07 17:14:41,338 | server.py:101 | FL starting
INFO flower 2022-12-07 17:14:41,339 | server.py:213 | fit_round 1: no clients selected, cancel
INFO flower 2022-12-07 17:14:41,340 | server.py:163 | evaluate_round 1: no clients selected, cancel
INFO flower 2022-12-07 17:14:41,341 | server.py:213 | fit_round 2: n

## Client and NumPyClient
In previous parts of this tutorial, we’ve based our client on NumPyClient, a convenience class which makes it easy to work with machine learning libraries that have good NumPy interoperability.
With Client, we gain a lot of flexibility that we didn’t have before, but we’ll also have to do a few things the we didn’t have to do before.

# Step 1: Revisiting NumPyClient
We’ve seen this before, there’s nothing new so far. The only tiny difference compared to the previous notebook is naming, we’ve changed FlowerClient to FlowerNumPyClient and client_fn to numpyclient_fn.

In [22]:
class FlowerNumPyClient(fl.client.NumPyClient):
    def __init__(self, cid, net, trainloader, valloader):
        self.cid = cid
        self.net = net
        self.trainloader = trainloader
        self.valloader = valloader

    def get_parameters(self, config):
        print(f"[Client {self.cid}] get_parameters")
        return get_parameters(self.net)

    def fit(self, parameters, config):
        print(f"[Client {self.cid}] fit, config: {config}")
        set_parameters(self.net, parameters)
        train(self.net, self.trainloader, epochs=1)
        return get_parameters(self.net), len(self.trainloader), {}

    def evaluate(self, parameters, config):
        print(f"[Client {self.cid}] evaluate, config: {config}")
        set_parameters(self.net, parameters)
        loss, accuracy = test(self.net, self.valloader)
        return float(loss), len(self.valloader), {"accuracy": float(accuracy)}


def numpyclient_fn(cid) -> FlowerNumPyClient:
    net = Net().to(DEVICE)
    trainloader = trainloaders[int(cid)]
    valloader = valloaders[int(cid)]
    return FlowerNumPyClient(cid, net, trainloader, valloader)

In [23]:
fl.simulation.start_simulation(
    client_fn=numpyclient_fn,
    num_clients=2,
    config=fl.server.ServerConfig(num_rounds=3),
)

INFO flower 2022-12-07 17:19:41,493 | app.py:140 | Starting Flower simulation, config: ServerConfig(num_rounds=3, round_timeout=None)
2022-12-07 17:19:45,418	INFO worker.py:1518 -- Started a local Ray instance.
INFO flower 2022-12-07 17:19:46,737 | app.py:174 | Flower VCE: Ray initialized with resources: {'object_store_memory': 2147483648.0, 'node:127.0.0.1': 1.0, 'CPU': 8.0, 'memory': 17722780058.0}
INFO flower 2022-12-07 17:19:46,738 | server.py:86 | Initializing global parameters
INFO flower 2022-12-07 17:19:46,739 | server.py:270 | Requesting initial parameters from one random client
INFO flower 2022-12-07 17:19:49,462 | server.py:274 | Received initial parameters from one random client
INFO flower 2022-12-07 17:19:49,463 | server.py:88 | Evaluating initial parameters
INFO flower 2022-12-07 17:19:49,463 | server.py:101 | FL starting
DEBUG flower 2022-12-07 17:19:49,464 | server.py:215 | fit_round 1: strategy sampled 2 clients (out of 2)


(launch_and_get_parameters pid=10133) [Client 0] get_parameters
(launch_and_fit pid=10133) [Client 1] fit, config: {}
(launch_and_fit pid=10134) [Client 0] fit, config: {}
(launch_and_fit pid=10133) Epoch 1: train loss 0.06601455807685852, accuracy 0.20644444444444446


DEBUG flower 2022-12-07 17:19:54,675 | server.py:229 | fit_round 1 received 2 results and 0 failures
WARNING flower 2022-12-07 17:19:54,681 | fedavg.py:242 | No fit_metrics_aggregation_fn provided
DEBUG flower 2022-12-07 17:19:54,682 | server.py:165 | evaluate_round 1: strategy sampled 2 clients (out of 2)


(launch_and_fit pid=10134) Epoch 1: train loss 0.06409581750631332, accuracy 0.23555555555555555


DEBUG flower 2022-12-07 17:19:57,290 | server.py:179 | evaluate_round 1 received 2 results and 0 failures
WARNING flower 2022-12-07 17:19:57,290 | fedavg.py:273 | No evaluate_metrics_aggregation_fn provided
DEBUG flower 2022-12-07 17:19:57,291 | server.py:215 | fit_round 2: strategy sampled 2 clients (out of 2)


...

DEBUG flower 2022-12-07 17:20:07,664 | server.py:229 | fit_round 3 received 2 results and 0 failures
DEBUG flower 2022-12-07 17:20:07,670 | server.py:165 | evaluate_round 3: strategy sampled 2 clients (out of 2)


(launch_and_fit pid=10133) Epoch 1: train loss 0.05136294290423393, accuracy 0.40066666666666667
(launch_and_fit pid=10134) Epoch 1: train loss 0.05197069048881531, accuracy 0.3948888888888889


DEBUG flower 2022-12-07 17:20:10,055 | server.py:179 | evaluate_round 3 received 2 results and 0 failures
INFO flower 2022-12-07 17:20:10,055 | server.py:144 | FL finished in 20.591251875000125
INFO flower 2022-12-07 17:20:10,056 | app.py:192 | app_fit: losses_distributed [(1, 0.06073317456245422), (2, 0.055544139623641964), (3, 0.05297538387775422)]
INFO flower 2022-12-07 17:20:10,057 | app.py:193 | app_fit: metrics_distributed {}
INFO flower 2022-12-07 17:20:10,057 | app.py:194 | app_fit: losses_centralized []
INFO flower 2022-12-07 17:20:10,057 | app.py:195 | app_fit: metrics_centralized {}


(launch_and_evaluate pid=10134) [Client 0] evaluate, config: {}
(launch_and_evaluate pid=10133) [Client 1] evaluate, config: {}


History (loss, distributed):
	round 1: 0.06073317456245422
	round 2: 0.055544139623641964
	round 3: 0.05297538387775422

Let’s dive a little bit deeper and discuss how Flower executes this simulation. Whenever a client is selected to do some work, start_simulation calls the function numpyclient_fn to create an instance of our FlowerNumPyClient (along with loading the model and the data).

But here’s the perhaps surprising part: Flower doesn’t actually use the FlowerNumPyClient object directly. Instead, it wraps the object to makes it look like a subclass of flwr.client.Client, not flwr.client.NumPyClient.
 In fact, the Flower core framework **doesn’t know how to handle NumPyClient’s, it only knows how to handle Client’s**. NumPyClient is just a convenience abstraction built on top of Client.

# Step 2: Moving from NumPyClient to Client

In [24]:
from flwr.common import Code, EvaluateIns, EvaluateRes, FitIns, FitRes, GetParametersIns, GetParametersRes, Status
from flwr.common import ndarrays_to_parameters, parameters_to_ndarrays


class FlowerClient(fl.client.Client):
    def __init__(self, cid, net, trainloader, valloader):
        self.cid = cid
        self.net = net
        self.trainloader = trainloader
        self.valloader = valloader

    def get_parameters(self, ins: GetParametersIns) -> GetParametersRes:
        print(f"[Client {self.cid}] get_parameters")

        # Get parameters as a list of NumPy ndarray's
        ndarrays: List[np.ndarray] = get_parameters(self.net)

        # Serialize ndarray's into a Parameters object
        parameters = ndarrays_to_parameters(ndarrays)

        # Build and return response
        status = Status(code=Code.OK, message="Success")
        return GetParametersRes(
            status=status,
            parameters=parameters,
        )

    def fit(self, ins: FitIns) -> FitRes:
        print(f"[Client {self.cid}] fit, config: {ins.config}")

        # Deserialize parameters to NumPy ndarray's
        parameters_original = ins.parameters
        ndarrays_original = parameters_to_ndarrays(parameters_original)

        # Update local model, train, get updated parameters
        set_parameters(self.net, ndarrays_original)
        train(self.net, self.trainloader, epochs=1)
        ndarrays_updated = get_parameters(self.net)

        # Serialize ndarray's into a Parameters object
        parameters_updated = ndarrays_to_parameters(ndarrays_updated)

        # Build and return response
        status = Status(code=Code.OK, message="Success")
        return FitRes(
            status=status,
            parameters=parameters_updated,
            num_examples=len(self.trainloader),
            metrics={},
        )

    def evaluate(self, ins: EvaluateIns) -> EvaluateRes:
        print(f"[Client {self.cid}] evaluate, config: {config}")

        # Deserialize parameters to NumPy ndarray's
        parameters_original = ins.parameters
        ndarrays_original = parameters_to_ndarrays(parameters_original)

        set_parameters(self.net, ndarrays_original)
        loss, accuracy = test(self.net, self.valloader)
        # return float(loss), len(self.valloader), {"accuracy": float(accuracy)}

        # Build and return response
        status = Status(code=Code.OK, message="Success")
        return EvaluateRes(
            status=status,
            loss=float(loss),
            num_examples=len(self.valloader),
            metrics={"accuracy": float(accuracy)},
        )


def client_fn(cid) -> FlowerClient:
    net = Net().to(DEVICE)
    trainloader = trainloaders[int(cid)]
    valloader = valloaders[int(cid)]
    return FlowerClient(cid, net, trainloader, valloader)

In [25]:
fl.simulation.start_simulation(
    client_fn=numpyclient_fn,
    num_clients=2,
    config=fl.server.ServerConfig(num_rounds=3),
)

INFO flower 2022-12-07 17:22:46,721 | app.py:140 | Starting Flower simulation, config: ServerConfig(num_rounds=3, round_timeout=None)
2022-12-07 17:22:50,529	INFO worker.py:1518 -- Started a local Ray instance.
INFO flower 2022-12-07 17:22:52,070 | app.py:174 | Flower VCE: Ray initialized with resources: {'CPU': 8.0, 'node:127.0.0.1': 1.0, 'memory': 17762622669.0, 'object_store_memory': 2147483648.0}
INFO flower 2022-12-07 17:22:52,071 | server.py:86 | Initializing global parameters
INFO flower 2022-12-07 17:22:52,072 | server.py:270 | Requesting initial parameters from one random client
INFO flower 2022-12-07 17:22:54,322 | server.py:274 | Received initial parameters from one random client
INFO flower 2022-12-07 17:22:54,323 | server.py:88 | Evaluating initial parameters
INFO flower 2022-12-07 17:22:54,323 | server.py:101 | FL starting
DEBUG flower 2022-12-07 17:22:54,324 | server.py:215 | fit_round 1: strategy sampled 2 clients (out of 2)


(launch_and_get_parameters pid=10155) [Client 1] get_parameters
(launch_and_fit pid=10155) [Client 1] fit, config: {}
(launch_and_fit pid=10156) [Client 0] fit, config: {}
(launch_and_fit pid=10155) Epoch 1: train loss 0.06513454020023346, accuracy 0.226


DEBUG flower 2022-12-07 17:22:59,964 | server.py:229 | fit_round 1 received 2 results and 0 failures
WARNING flower 2022-12-07 17:22:59,970 | fedavg.py:242 | No fit_metrics_aggregation_fn provided
DEBUG flower 2022-12-07 17:22:59,970 | server.py:165 | evaluate_round 1: strategy sampled 2 clients (out of 2)


(launch_and_fit pid=10156) Epoch 1: train loss 0.06510709971189499, accuracy 0.2157777777777778


DEBUG flower 2022-12-07 17:23:02,229 | server.py:179 | evaluate_round 1 received 2 results and 0 failures
WARNING flower 2022-12-07 17:23:02,230 | fedavg.py:273 | No evaluate_metrics_aggregation_fn provided
DEBUG flower 2022-12-07 17:23:02,230 | server.py:215 | fit_round 2: strategy sampled 2 clients (out of 2)


...

DEBUG flower 2022-12-07 17:23:12,367 | server.py:229 | fit_round 3 received 2 results and 0 failures
DEBUG flower 2022-12-07 17:23:12,372 | server.py:165 | evaluate_round 3: strategy sampled 2 clients (out of 2)


(launch_and_fit pid=10155) Epoch 1: train loss 0.05398540571331978, accuracy 0.37444444444444447
(launch_and_fit pid=10156) Epoch 1: train loss 0.05347144231200218, accuracy 0.3731111111111111


DEBUG flower 2022-12-07 17:23:14,568 | server.py:179 | evaluate_round 3 received 2 results and 0 failures
INFO flower 2022-12-07 17:23:14,569 | server.py:144 | FL finished in 20.244424930000605
INFO flower 2022-12-07 17:23:14,569 | app.py:192 | app_fit: losses_distributed [(1, 0.06234578847885132), (2, 0.057087348222732544), (3, 0.05480945611000061)]
INFO flower 2022-12-07 17:23:14,570 | app.py:193 | app_fit: metrics_distributed {}
INFO flower 2022-12-07 17:23:14,570 | app.py:194 | app_fit: losses_centralized []
INFO flower 2022-12-07 17:23:14,570 | app.py:195 | app_fit: metrics_centralized {}


(launch_and_evaluate pid=10156) [Client 0] evaluate, config: {}
(launch_and_evaluate pid=10155) [Client 1] evaluate, config: {}


History (loss, distributed):
	round 1: 0.06234578847885132
	round 2: 0.057087348222732544
	round 3: 0.05480945611000061

## Why Client instead of Nump.Client?
The difference comes from the fact that Client expects us to take care of parameter serialization and deserialization.

For Flower to be able to send parameters over the network, it eventually needs to turn these parameters into bytes.
Turning parameters (e.g., NumPy ndarray’s) into raw bytes is called serialization.
Turning raw bytes into something more useful (like NumPy ndarray’s) is called deserialization.
Flower needs to do both: it needs to serialize parameters on the server-side and send them to the client,
the client needs to deserialize them to use them for local training,
and then serialize the updated parameters again to send them back to the server, which (finally!) deserializes them again in order to aggregate them with the updates received from other clients.



The only real difference between Client and NumPyClient is that NumPyClient takes care of serialization and deserialization for you.
It can do so because it expects you to return parameters as NumPy ndarray’s, and it knows how to handle these. This makes working with machine learning libraries that have good NumPy support (most of them) a breeze.

In terms of API, there’s one major difference: all methods in Client take exectly one argument (e.g., FitIns in Client.fit) and return exactly one value (e.g., FitRes in Client.fit).
The methods in NumPyClient on the other hand have multiple arguments (e.g., parameters and config in NumPyClient.fit) and multiple return values
(e.g., parameters, num_example, and metrics in NumPyClient.fit) if there are multiple things to handle. These *Ins and *Res objects in Client wrap all the individual values you’re used to from NumPyClient.